# Sistemas Distribuidos de Procesamiento de Datos II

# Práctica final T2 (Parte I)

• Datos: Fichero con tweets del GP MotoGP de Qatar 2014 (ALTO DATABASE).

• Tecnologías: Spark SQL, DataFrames, Jupyter notebook.

• Versión Spark: 2.2.1 o superior.

• Fecha de entrega: Domingo, 6 de mayo de 2018, a las 23:55.

## Librerias

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = (SparkSession.builder
    .master("local[*]")
    .config("spark.driver.cores", 1)
    .appName("Practica SparkSQL MotoGP 2014")
    .getOrCreate() )
sc = spark.sparkContext
print(spark)
print(sc)

<SparkContext master=local[*] appName=Practica SparkSQL MotoGP 2014>


In [2]:
    from pyspark.sql.types import *
    from pyspark.sql.functions import *

## Importamos los datos

Primero definimos el esquema de datos

In [3]:
customSchema = StructType([StructField("Id", LongType(), True),
                           StructField("Parent_sys_id", StringType(), True),
                           StructField("Source", StringType(), True),
                           StructField("Mentions", StringType(), True),
                           StructField("Target", StringType(), True),
                           StructField("Name_source", StringType(), True),
                           StructField("Body", StringType(), True),
                           StructField("Pub_date", TimestampType(), True),
                           StructField("URLs", StringType(), True),
                           StructField("Tipe_action", StringType(), True),
                           StructField("Link", StringType(), True),
                           StructField("Has_link", ByteType(), True),
                           StructField("Has_picture", ByteType(), True),
                           StructField("Website", StringType(), True),
                           StructField("Country", StringType(), True),
                           StructField("Activity", LongType(), True),
                           StructField("Followers", LongType(), True),
                           StructField("Following", LongType(), True),
                           StructField("Location", StringType(), True)
                          ])

A continuación cargamos los datos utilizando el esquema definido:

In [4]:
events = spark.read.csv("data/DATASET-Twitter-23-26-Mar-2014-MotoGP-Qatar.csv",
                        header=True, schema=customSchema, timestampFormat="dd/MM/yyyy HH:mm")
                        #mode="FAILFAST") #

In [5]:
events.count()

257680

Se han cargaado 257680 registros.

Comprobamos que el esquema del dataframe se corresponde con el definido:

In [6]:
events.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Parent_sys_id: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Mentions: string (nullable = true)
 |-- Target: string (nullable = true)
 |-- Name_source: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Pub_date: timestamp (nullable = true)
 |-- URLs: string (nullable = true)
 |-- Tipe_action: string (nullable = true)
 |-- Link: string (nullable = true)
 |-- Has_link: byte (nullable = true)
 |-- Has_picture: byte (nullable = true)
 |-- Website: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Activity: long (nullable = true)
 |-- Followers: long (nullable = true)
 |-- Following: long (nullable = true)
 |-- Location: string (nullable = true)



## PREGUNTA A

a) Calcular el número total de retweets por usuario para los 50 usuarios con más mensajes
en la muestra de tweets analizados. Calcular, para cada uno de estos usuarios la media de
enlaces (URLs) enviados por mensaje. (2.5 puntos).

Los 50 usuarios con más mensajes de la muestra:

In [ ]:
users_50 = events\
 .groupBy("Source")\
 .agg(count("Id").alias("tweets"))\
 .orderBy("tweets", ascending=False)\
 .limit(50)
users_50.show(50)

Numero total de retweets por usuario para los 50 usuarios con mas mensajes en la muestra:

In [ ]:
events.where(events.Tipe_action == "RT")\
.join(users_50,"Source", how="right")\
.groupBy("Source","Tipe_action")\
.agg(count("Id").alias("retweets"))\
.orderBy("retweets", ascending=False)\
.show(50) # Son 44

Hay seis twitteros de los 50 con mas mensajes de la muestra que no han hecho ningun retweet.

Media de enlaces URL enviados por mensaje para los 50 usuarios con mas mensajes en la muestra:

In [ ]:
# Creamos una UDF para obtener el número de URLs por mensaje:
def string_len_list(string):
    if string is not None and string != '0':
        return len(string.split(','))
    else:
        return 0
string_len_list_udf = udf(string_len_list, IntegerType())

Calculamos la media de URLs sumando el numero total de URL´s incluidas en todos los tweets (TW, RT y MT) de un usuario dividido entre el número total de tweets enviados por dicho usuario:

In [ ]:
events.join(users_50,"Source", how = "right")\
.withColumn("numero_URLs", string_len_list_udf(events.URLs))\
.groupBy("Source")\
.agg(sum("numero_URLs").alias("total_URLs"),count("Id").alias("tweets"),(round(sum("numero_URLs")/count("Id"),2)).alias("media_URLs"))\
.orderBy("media_URLs", ascending=False)\
.show(50)

## PREGUNTA B

b) Calcular el número total de mensajes que contienen información de geolocalización en
el campo LOCATION.(2.5 puntos).

Si la variable de Gelocalización "Location" empieza por “ÜT”, esto indica que se proporcionan las coordenadas exactas desde donde se emitió el tweet.

In [ ]:
events.where(col("Location").like("ÜT%")).count()

Observamos que hay 5 twitts a mayores que contienen "ÜT" en la variable "Location" pero no necesariamente al comienzo de la misma:

In [ ]:
events.where(col("Location").like("%ÜT%")).count() - events.where(col("Location").like("ÜT%")).count()

## PREGUNTA C

c) Calcular las 10 cuentas de Twitter que más han sido mencionadas en todo el conjunto de
datos analizados. (2.5 puntos).

En el campo "Mentions" aparecen los usuarios separados por comas presentes en el contenido de aquellos mensajes tipo RT (Retweet) o MT (Mención).

In [ ]:
events.where(events.Tipe_action != "TW")\
.select(explode(split(col("Mentions"), ",")).alias("cuentas_mencionadas"))\
.groupBy("cuentas_mencionadas")\
.agg(count("cuentas_mencionadas").alias("tweets"))\
.orderBy("tweets", ascending=False)\
.limit(10).show()

## PREGUNTA D

d) Calcular los 10 mensajes más retweeteados y los 10 mensajes que han acumulado más
respuestas en la muestra de datos analizados. Ahora, restringe la búsqueda a los mensajes
en el intervalo 2014-03-24 04:00 - 2014-03-24 10:00. (2.5 puntos).

El campo "Parent_sys_id", si no es nulo ("sin padre"), contiene un ID que relaciona el post con aquel del cual depende por ser un retweet o una respuesta.

Los diez mensajes más retwiteados:

In [ ]:
mensajes_RT_10 = events.where((events.Tipe_action == "RT") & (events.Parent_sys_id != "sin padre"))\
.groupBy("Parent_sys_id")\
.agg(count("Parent_sys_id").alias("tweets"))\
.orderBy("tweets", ascending=False)\
.withColumnRenamed("Parent_sys_id", "Id")\
.limit(10)
mensajes_RT_10.show()

Hacemos un join por "Id" con el dataframe original "events" para obtener más información acerca de los 10 mensajes mas retweeteados:

In [ ]:
events.join(mensajes_RT_10, on = "Id", how="right")\
.select("Id","Source","Body")\
.orderBy("tweets", ascending=False).take(10)

Los diez mensajes que han acumulado más respuestas:

In [ ]:
mensajes_MT_10 = events.where((events.Tipe_action == "MT")& (events.Parent_sys_id != "sin padre"))\
.groupBy("Parent_sys_id")\
.agg(count("Parent_sys_id").alias("tweets"))\
.orderBy("tweets", ascending=False)\
.withColumnRenamed("Parent_sys_id", "Id")\
.limit(10)
mensajes_MT_10.show()

Hacemos un join por "Id" con el dataframe original "events" para obtener más información acerca de los 10 mensajes que han acumulado un mayor numero de respuestas:

In [ ]:
events.join(mensajes_MT_10, on = "Id", how="right")\
.select("Id","Source","Body")\
.orderBy("tweets", ascending=False).take(10)

Restringir la busqueda a los mensajes en el intervalo 2014-03-24 04:00 - 2014-03-24 10:00:

El campo "Pub_date" contiene la fecha y hora en la que se ha escrito el mensaje.

In [ ]:
mensajes_RT_10_date = events.where(events.Pub_date.between('2014-03-24 04:00' , '2014-03-24 10:00'))\
.where((events.Tipe_action == "RT") & (events.Parent_sys_id != "sin padre"))\
.groupBy("Parent_sys_id")\
.agg(count("Parent_sys_id").alias("tweets"))\
.orderBy("tweets", ascending=False)\
.withColumnRenamed("Parent_sys_id", "Id")\
.limit(10)
mensajes_RT_10_date.show()

In [ ]:
events.join(mensajes_RT_10_date, on = "Id", how="right")\
.select("Id","Source","Body")\
.orderBy("tweets", ascending=False).take(10)

In [ ]:
mensajes_MT_10_date = events.where(events.Pub_date.between('2014-03-24 04:00' , '2014-03-24 10:00'))\
.where((events.Tipe_action == "MT")& (events.Parent_sys_id != "sin padre"))\
.groupBy("Parent_sys_id")\
.agg(count("Parent_sys_id").alias("tweets"))\
.orderBy("tweets", ascending=False)\
.withColumnRenamed("Parent_sys_id", "Id")\
.limit(10)
mensajes_MT_10_date.show()

In [ ]:
events.join(mensajes_MT_10_date, on = "Id", how="right")\
.select("Id","Source","Body")\
.orderBy("tweets", ascending=False).take(10)

## Paramos el contexto de Spark

In [ ]:
sc.stop()